In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import pudl
import pudl.constants as pc
import sqlalchemy as sa
import logging
import pathlib
from copy import deepcopy
import sys

from pudl_rmi.deprish import *
from pudl_rmi import connect_deprish_to_eia

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)

In [ ]:
pudl_settings = pudl.workspace.setup.get_defaults()
ferc1_engine = sa.create_engine(pudl_settings["ferc1_db"])
pudl_engine = sa.create_engine(pudl_settings["pudl_db"])

In [ ]:
pudl_out = pudl.output.pudltabl.PudlTabl(
    pudl_engine,freq='AS',
    fill_fuel_cost=True,
    roll_fuel_cost=True,
    fill_net_gen=False
)

### Process Depreciation Data

In [ ]:
file_path_deprish = pathlib.Path().cwd().parent/'inputs'/'depreciation_rmi.xlsx'

# get the studies from the spreadsheet
extract_df = Extractor(
    file_path=file_path_deprish,
    sheet_name=0).execute()

# create a transformer object to process the extracted data
transformer = Transformer(
    extract_df=extract_df
)

In [ ]:
# the transformer has 3 main steps that are encapsulated in execute
#   1. tidying w/ early_tidy()
#   2. reshaping w/ reshape() - this is where the common plant allocaiton is happening
#   3. filling-in w/ fill_in()
#   4. aggregating by plant w/ agg_to_idx()
# Note: two warnings will be thrown when you run this rn.
# there are the remaning issues in the common plant allocation process
# that need to be ironed out
transform_df = transformer.execute(clobber=True)

In [ ]:
path_deprish =  pathlib.Path().cwd().parent/'outputs' /'deprish_cleaned.pkl.gz'
transform_df.to_pickle(path_deprish)

### Export

In [ ]:
path_common_dc =  pathlib.Path().cwd().parent/'outputs' /'deprish_cleaned.xlsx'
connect_deprish_to_eia.save_to_workbook(
    path_common_dc,
    sheets_df_dict={
        'deprish_cleaned': transform_df,
        'deprish_cleaned_w_ferc_acct': transformer.reshaped_df,
        'deprish_raw': extract_df
    },
)

### Auto Generate Common Associations

In [ ]:
# file_path_deprish = pathlib.Path().cwd().parent/'inputs'/'depreciation_rmi.xlsx'
#common_assn = make_default_common_assn(file_path_deprish)

In [ ]:
# path_common_assn=  pathlib.Path().cwd().parent/'outputs' /'common_assn.csv'
# common_assn.to_csv(path_common_assn,index=False)